In [14]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, FullScreenControl, DrawControl, Marker, \
                        Circle, LayersControl, Polyline, Rectangle, WidgetControl, LayerGroup, AwesomeIcon, Icon, ScaleControl, GeoJSON
from ipywidgets import IntSlider, FloatSlider, VBox, HBox, Tab, Button, Dropdown, Text, Layout, IntRangeSlider, Checkbox, Label, RadioButtons, HTML, FloatText, ToggleButtons
import math
import aerocalc3
import aerocalc3.unit_conversion as convert
import aerocalc3.std_atm
import aerocalc3.airspeed
import re

import pyproj

geo = pyproj.Geod(ellps="WGS84")
constant_g = 9.80665

user_lat = 15.0
user_long = -15.0

In [15]:

m = Map(
    # basemap=basemaps.OpenTopoMap,
    basemap=basemaps.GeoportailFrance.orthos,
    center=(46.5, 6.5),
    scroll_wheel_zoom=True,
    # double_click_zoom=False,
    zoom=5
)

m.add_control(FullScreenControl())

ee_basemaps = {}

# Loops through all ipyleaflet basemaps

for item in basemaps.values():
#     print(item.get("name", "No name."))
#     print(item.get("url", "No url."))
#     print(item.keys())
    try:
        name = item["name"]
        basemap = "basemaps.{}".format(name)
        ee_basemaps[name] = basemap_to_tiles(eval(basemap))
    except:
        try:
            for sub_item in item:
                name = item[sub_item]["name"]
                basemap = "basemaps.{}".format(name)
                basemap = basemap.replace("Mids", "Modis")
                ee_basemaps[name] = basemap_to_tiles(eval(basemap))
        except:
            pass

# Adds a Dropdown widget
dropdown = Dropdown(
    options=list(ee_basemaps.keys()),
    value="GeoportailFrance.orthos",
    description="Basemaps",
)

# Handles Dropdown control event
def on_click(change):
    basemap_name = change["new"]
    old_basemap = m.layers[-1]
    m.substitute_layer(old_basemap, ee_basemaps[basemap_name])

dropdown.observe(on_click, "value")

# Adds control to the map
basemap_control = WidgetControl(widget=dropdown, position="topright")
m.add_control(basemap_control)

# control = LayersControl(position='topleft')
# m.add_control(control)

# maison = Marker(name='Maison', location=(43.559837, 1.314936))
# m.add_layer(maison)

# draw_control = DrawControl()
# draw_control.polyline =  {
#     "shapeOptions": {
#         "color": "purple",
#         "weight": 10,
#         "opacity": 0.2
#     }
# }
# draw_control.polygon = {}
# draw_control.circlemarker = {}
# draw_control.rectangle = {}

# m.add_control(draw_control)
m.add_control(ScaleControl(position='bottomleft', imperial = False, max_width = 150))



In [16]:
def _lat_label_format(option):

    if option == "Deg.dec":
        return "+/-DD.ddd"
    elif option == "DegMin.dec":
        return "N/SDDMM.ddd"
            
    elif option == "DegMinSec.dec":
        return "N/SDDMMSS.ddd"
    elif option == "Cursor position":
        return "+/-DD.ddd"
    else:
        return "Not defined"

def _lon_label_format(option):

    if option == "Deg.dec":
        return "+/-DDD.ddd"
    elif option == "DegMin.dec":
        return "W/EDDDMM.ddd"
            
    elif option == "DegMinSec.dec":
        return "W/EDDDMMSS.ddd"
    elif option == "Cursor position":
        return "+/-DDD.ddd"
    else:
        return "Not defined"

geo_choice = RadioButtons(options = ["Cursor position", "Deg.dec", "DegMin.dec", "DegMinSec.dec"], horizontal=True)
geo_format = Label(value= "Lat : " +  _lat_label_format(geo_choice.value) + ', Lon : ' + _lon_label_format(geo_choice.value))

lat_input = Text(description = "Lat : ", placeholder =_lat_label_format(geo_choice.value))
lon_input = Text(description = "Lon : ", placeholder =_lon_label_format(geo_choice.value))
go_button = Button(description = "GO !")


def label_format(option):
    geo_format.value = "Lat : " + _lat_label_format(option.new) + ", Lon : " + _lon_label_format(option.new) 
    lat_input.value = ''
    lon_input.value = ''
    lat_input.placeholder = _lat_label_format(option.new)
    lon_input.placeholder = _lon_label_format(option.new)
    
    
geo_choice.observe(label_format, "value")

# input_control = WidgetControl(widget = VBox([geo_choice, geo_format, lat_input, lon_input, go_button]), position="topright")
# m.add_control(input_control)


In [17]:
import pandas as pd
from pathlib import Path
ourairports_runways = "https://davidmegginson.github.io/ourairports-data/runways.csv" 
data_dir = Path("../data")
data_dir.mkdir(parents=True, exist_ok=True)
ourairports_dir = data_dir / "ourairports"
ourairports_dir.mkdir(parents=True, exist_ok=True)

# class runway():
#     def __init__(self).

def on_eosid_prop_change():
    pass

class runways():
    def __init__(self, **args):

        self.rwy_df = pd.DataFrame(columns=["surface","closed","le_ident", "he_ident", "length_ft", "airport_ident", "le_latitude_deg", "le_longitude_deg", 'he_latitude_deg', "he_longitude_deg", "heading_degT", "elevation_ft" ])
        self.rwy_draw = None
        self.dep_axis = None
        self.dep = None
        
        self.rwys_display = Polyline(color="purple", fill=False, weight = 5)
        m.add_layer(self.rwys_display)
        
        self.load_btn = Button(tooltip="Download data", icon = 'download', layout=Layout(width = "1cm"))
        self.ap_t = Text(placeholder="ICAO", layout=Layout(width = "1.5cm"))
        self.ok_btn = Button(icon="check", layout=Layout(width = "1cm"))
        self.rwy_dd = Dropdown(placeholder="RWY", layout=Layout(width = "2.5cm"))

        m.add_control(WidgetControl(widget = HBox([self.load_btn, self.ap_t, self.ok_btn, self.rwy_dd]), position = "topright"))
        
        self.load_btn.on_click(self.load_fct)
        self.ok_btn.on_click(self.ok_fct)
        self.rwy_dd.observe(self.display_one)
        
        if (ourairports_dir / "runways.csv").exists():
            self.rwy_df = pd.read_csv(ourairports_dir / "runways.csv")
            self.load_btn.button_style = "success"
        else:
            self.load_btn.button_style = "danger"
        self.init_runways()

    def init_runways(self):
        if not self.rwy_df.empty:
            self.rwy_df = self.rwy_df[self.rwy_df['surface'].isin(['CON', 'ASP'])]
            self.rwy_df = self.rwy_df[self.rwy_df['closed'] == 0]

            self.rwy_df["_rwy"] = self.rwy_df["le_ident"] + "/" + self.rwy_df["he_ident"]   
            self.rwy_df['_length'] = self.rwy_df['length_ft'] * 0.3048
        
    def load_fct(self, event = None):
        self.load_btn.button_style = ""
        self.load_btn.disabled = True
        self.rwy_df = pd.read_csv(ourairports_runways)
        self.rwy_df.to_csv(ourairports_dir / "runways.csv", index=False)
        self.load_btn.disabled = False
        self.init_runways()

    def ok_fct(self, event = None):
        if self.rwy_draw:
            m.remove_layer(self.rwy_draw)
            self.rwy_draw = None
            
        if len(self.ap_t.value) == 4:
            self.selected_rwys = self.rwy_df[self.rwy_df["airport_ident"] == self.ap_t.value.upper()].drop_duplicates()
            
            if not self.selected_rwys.empty:
                # a = self.selected_rwys["le_ident"].append(self.selected_rwys["he_ident"]).to_list()
                a = pd.concat([self.selected_rwys["le_ident"], self.selected_rwys["he_ident"]]).to_list()
                self.rwy_dd.options = a

                r = self.selected_rwys.reset_index(drop=True).iloc[0]
                m.center = (r["le_latitude_deg"], r["le_longitude_deg"])

                self._display_one(a[0])
                
            else:
                self.ap_t.value = ""
                
    def get_runways(self, **args):

        if args['type'] == "mouseup":
            self._get_runways()
            
    def _get_runways(self, event = None):      
#             print(m.bounds)
            min_lat = m.bounds[0][0]
            min_lon = m.bounds[0][1]
            max_lat = m.bounds[1][0]
            max_lon = m.bounds[1][1]
            
            extract = self.rwy_df[(self.rwy_df['le_latitude_deg'] >= min_lat) & (self.rwy_df['he_latitude_deg'] <= max_lat) &
                        (self.rwy_df['le_longitude_deg'] >= min_lon) & (self.rwy_df['he_longitude_deg'] <= max_lon)]
                        
            locations = list()
            def _line(r):                
                locations.append(((r["le_latitude_deg"], r["le_longitude_deg"]), (r["he_latitude_deg"], r["he_longitude_deg"])))

            extract.apply(_line, axis = 1)

            self.rwys_display.locations = locations
            
    def _display_one(self, rwy):
        if self.rwy_draw:
            m.remove_layer(self.rwy_draw)
            self.rwy_draw = None
        r = self.rwy_df[(self.rwy_df['le_ident'] == rwy) & (self.rwy_df['airport_ident'] == self.ap_t.value.upper())] 
        th_prefix = "le_"
        opp_prefix = "he_"
        if r.empty:
            r = self.rwy_df[(self.rwy_df['he_ident'] == rwy) & (self.rwy_df['airport_ident'] == self.ap_t.value.upper())]
            th_prefix = "he_"
            opp_prefix = "le_"

        r = r.reset_index(drop=True).iloc[0]
#             print(r)
        ### %%%%%
        self.rwy_draw = Polyline(locations = ((r["le_latitude_deg"], r["le_longitude_deg"]), (r["he_latitude_deg"], r["he_longitude_deg"]) )  , color="darkgreen", fill=False, weight = 10) 
        m.add_layer(self.rwy_draw) 
            
    def display_one(self, event):
#         print(event)
        if event["name"] == "label":



            self._display_one(event['new'])
            
            
rwy = runways()
# m.on_interaction(rwy.get_runways)
m.observe(rwy._get_runways, "bounds")

In [18]:
last_point_location = None

speed_kts = IntRangeSlider(value = [440, 460], min = 150, max = 500, step = 1, 
                 description = "Speed (kts)", orientation='horizontal', readout=True, readout_format="d")
duration_sec = IntRangeSlider(value = [60, 67], min = 0, max = 900, step = 1, 
                 description = "Duration (s)", orientation='horizontal', readout=True, readout_format="d")

c_ms_md = Circle(color = "green", weight = 3, fill = False)
c_ms_Md = Circle(color = "yellow", weight = 3, fill = False)
c_Ms_md = Circle(color = "orange", weight = 3, fill = False)
c_Ms_Md = Circle(color = "red", weight = 3, fill = False)
m.add_layer(LayerGroup(layers=(c_ms_md, c_ms_Md, c_Ms_md, c_Ms_Md)))

miniRange = Text(placeholder="MiniRange (NM)", description='Dist m (NM) :', disabled=True)
maxiRange = Text(placeholder="MaxiRange (NM)", description='Dist M (NM) :', disabled=True)

def rangeProcessing(event = None):
    mini_sp, maxi_sp = speed_kts.value
    mini_dur, maxi_dur = duration_sec.value
    if last_point_location:
        c_ms_md.location = last_point_location
        c_ms_md.radius = int(convert.speed_conv(mini_sp, from_units='kt', to_units='m/s') * mini_dur)
        
        c_ms_Md.location = last_point_location
        c_ms_Md.radius = int(convert.speed_conv(mini_sp, from_units='kt', to_units='m/s') * maxi_dur)
        
        c_Ms_md.location = last_point_location
        c_Ms_md.radius = int(convert.speed_conv(maxi_sp, from_units='kt', to_units='m/s') * mini_dur)
        
        c_Ms_Md.location =last_point_location
        c_Ms_Md.radius = int(convert.speed_conv(maxi_sp, from_units='kt', to_units='m/s') * maxi_dur)
        
        miniRange.value = str(convert.len_conv(c_ms_md.radius, from_units='m', to_units='nm')) 
        maxiRange.value = str(convert.len_conv(c_Ms_Md.radius, from_units='m', to_units='nm'))  
        
speed_kts.observe(rangeProcessing, "value")  
duration_sec.observe(rangeProcessing, "value")  

In [19]:
gs_kts = IntSlider(value = 400, min = 150, max = 500, step = 1, 
                 description = "Speed (kts)", orientation='horizontal', readout=True, readout_format="d")
bank_angle = IntSlider(value = 25, min = 1, max = 40, step = 1, 
                 description = "Bank angle (°)", orientation='horizontal', readout=True, readout_format="d")

turn_circle = Circle(color = "magenta", weight = 2, fill = False)


m.add_layer(turn_circle)

radius_nm = Text(placeholder="Turn radius (NM)", description='Radius (NM) :', disabled=True)

def radius_computing(event = None):
    gs = convert.speed_conv(gs_kts.value, from_units='kt', to_units='m/s')
    ba = math.radians(bank_angle.value)
    
    if last_point_location:
        turn_circle.location = last_point_location
        
        rad = math.pow(gs, 2) / (constant_g * math.tan(ba))
        turn_circle.radius = int(rad)
        
        radius_nm.value = str(convert.len_conv(turn_circle.radius, from_units='m', to_units='nm'))  
        
gs_kts.observe(radius_computing, "value")  
bank_angle.observe(radius_computing, "value")  

In [20]:
class std_atm():
    def __init__(self):
        self.press_alt_ft = IntSlider(value = 38000, min = 0, max = 45000, step = 100, 
                 description = "PA (ft)", orientation='horizontal', readout=True, readout_format="d")
        self.press_hpa = IntSlider(value = 1013, min = 0, max = 1013, step = 1, 
                 description = "Press (hpa)", orientation='horizontal', readout=True, readout_format="d")
        self.temp_deg = IntSlider(value = 15, min = -57, max = 15, step = 1, 
                 description = "Temp (deg)", orientation='horizontal', readout=True, readout_format="d")
        self.sound_speed = IntSlider(value = 300, min = 570, max = 665, step = 1, 
                 description = "Sound (kts)", orientation='horizontal', readout=True, readout_format="d")
        self.rho= FloatSlider(value = 1.15, min = 0, max = 1.225, step = 0.001, 
                 description = "Rho (kg/m3)", orientation='horizontal', readout=True, readout_format="f")
        
        
        self.from_press_alt_ft()
        self.observers()
        
    def observers(self):
        self.press_alt_ft.observe(self.from_press_alt_ft, "value")
        self.press_hpa.observe(self.from_press_hpa, "value")
        self.temp_deg.observe(self.from_temp_deg, "value")
        self.rho.observe(self.from_rho, "value")

    def unobservers(self):
        self.press_alt_ft.unobserve(self.from_press_alt_ft, "value")
        self.press_hpa.unobserve(self.from_press_hpa, "value")
        self.temp_deg.unobserve(self.from_temp_deg, "value")
        self.rho.unobserve(self.from_rho, "value")   
        
        
    def vbox(self):
        return VBox([self.press_alt_ft, self.press_hpa, self.temp_deg, self.sound_speed, self.rho])
    
    
    def from_press_alt_ft(self, event = None):
        self.unobservers()
        self.press_hpa.value = aerocalc3.std_atm.alt2press(self.press_alt_ft.value, alt_units='ft', press_units='hpa')
        self.temp_deg.value = aerocalc3.std_atm.alt2temp(self.press_alt_ft.value, alt_units="ft", temp_units="C")
        self.rho.value = aerocalc3.std_atm.alt2density(self.press_alt_ft.value, alt_units='ft', density_units='kg/m**3')
        self.sound_speed.value = int(aerocalc3.std_atm.temp2speed_of_sound(self.temp_deg.value, temp_units='C', speed_units='kt'))
        self.observers()
        
    def from_press_hpa(self, event = None):
        self.unobservers()
        self.press_alt_ft.value = aerocalc3.std_atm.press2alt(self.press_hpa.value, alt_units='ft', press_units='hpa')
        self.temp_deg.value = aerocalc3.std_atm.alt2temp(self.press_alt_ft.value, alt_units="ft", temp_units="C")
        self.rho.value = aerocalc3.std_atm.alt2density(self.press_alt_ft.value, alt_units='ft', density_units='kg/m**3') 
        self.sound_speed.value = aerocalc3.std_atm.temp2speed_of_sound(self.temp_deg.value, temp_units='C', speed_units='kt')
        self.observers()
        
    def from_temp_deg(self, event = None):
        self.unobservers()
        self.sound_speed.value = aerocalc3.std_atm.temp2speed_of_sound(self.temp_deg.value, temp_units='C', speed_units='kt')
        self.observers()
        
    def from_rho(self, event = None):   
        self.unobservers()
        self.press_alt_ft.value = aerocalc3.std_atm.density2alt(self.rho.value, alt_units='ft',density_units='kg/m**3')
        self.press_hpa.value = aerocalc3.std_atm.alt2press(self.press_alt_ft.value, alt_units='ft', press_units='hpa')
        self.temp_deg.value = aerocalc3.std_atm.alt2temp(self.press_alt_ft.value, alt_units="ft", temp_units="C")
        self.sound_speed.value = aerocalc3.std_atm.temp2speed_of_sound(self.temp_deg.value, temp_units='C', speed_units='kt')
        self.observers()
            
isa = std_atm()

In [21]:
# https://kilohotel.com/python/aerocalc/html/

class airspeed():
    def __init__(self):
        self.press_alt_ft = IntSlider(value = 38000, min = 0, max = 45000, step = 100, 
                 description = "PA (ft)", orientation='horizontal', readout=True, readout_format="d")
        self.press_hpa = IntSlider(value = 1013, min = 0, max = 1013, step = 1, 
                 description = "Press (hpa)", orientation='horizontal', readout=True, readout_format="d")
        
        self.fixed = ToggleButtons(options=['CAS', 'Mach', 'TAS'], style={"button_width": "2cm"})
        
        self.mach = FloatSlider(value = 0.85, min = 0, max = 1, step = 0.01, 
                 description = "Mach", orientation='horizontal', readout=True, readout_format=".2f")
        self.tas_kts = IntSlider(value = 300, min = 0, max = 661, step = 1, 
                 description = "TAS (kts)", orientation='horizontal', readout=True, readout_format="d")
        self.cas_kts = IntSlider(value = 300, min = 0, max = 661, step = 1, 
                 description = "CAS (kts)", orientation='horizontal', readout=True, readout_format="d")
        
        self.from_press_alt_ft()
        self.observers()
        
        
    def observers(self):
        self.press_alt_ft.observe(self.from_press_alt_ft, "value")
        self.press_hpa.observe(self.from_press_hpa, "value")
        self.mach.observe(self.from_mach, "value")
        self.tas_kts.observe(self.from_tas_kts, "value")
        self.cas_kts.observe(self.from_cas_kts, "value")
        
    def unobservers(self):
        self.press_alt_ft.unobserve(self.from_press_alt_ft, "value")
        self.press_hpa.unobserve(self.from_press_hpa, "value")
        self.mach.unobserve(self.from_mach, "value")
        self.tas_kts.unobserve(self.from_tas_kts, "value")
        self.cas_kts.unobserve(self.from_cas_kts, "value")
        
    def vbox(self):
        return VBox([self.press_alt_ft, self.press_hpa, self.fixed, self.mach, self.tas_kts, self.cas_kts])
    
    
    def from_press_alt_ft(self, event = None):
        self.unobservers()
        self.press_hpa.value = aerocalc3.std_atm.alt2press(self.press_alt_ft.value, alt_units='ft', press_units='hpa')
        self.__from()
        self.observers()
        
    def from_press_hpa(self, event = None):
        self.unobservers()
        self.press_alt_ft.value = aerocalc3.std_atm.press2alt(self.press_hpa.value, alt_units='ft', press_units='hpa')
        self.__from()
        self.observers()
        
    def __from(self):
        
        if self.fixed.value == "CAS":
            self.tas_kts.value = aerocalc3.airspeed.cas2tas(self.cas_kts.value, self.press_alt_ft.value, temp='std', speed_units='kt', alt_units='ft', temp_units='C')
            self.mach.value = aerocalc3.airspeed.cas_alt2mach(self.cas_kts.value, self.press_alt_ft.value, speed_units='kt', alt_units='ft')
            pass
            
        elif self.fixed.value == "Mach":
            self.cas_kts.value = aerocalc3.airspeed.mach_alt2cas(self.mach.value, self.press_alt_ft.value, alt_units='ft', speed_units='kt')
            self.tas_kts.value = aerocalc3.airspeed.mach2tas(self.mach.value, temp = 'std', altitude =  self.press_alt_ft.value, temp_units='C', alt_units='ft', speed_units='kt')
            
        elif self.fixed.value == "TAS":
            self.cas_kts.value = aerocalc3.airspeed.tas2cas(self.tas_kts.value, self.press_alt_ft.value, temp='std', speed_units='kt', alt_units='ft', temp_units='C')
            self.mach.value = aerocalc3.airspeed.tas2mach(self.tas_kts.value, temp='std', altitude = self.press_alt_ft.value, temp_units='C', alt_units='ft', speed_units='kt')
            
        
    def from_mach(self, event = None):
        self.unobservers()
        self.cas_kts.value = aerocalc3.airspeed.mach_alt2cas(self.mach.value, self.press_alt_ft.value, alt_units='ft', speed_units='kt')
        self.tas_kts.value = aerocalc3.airspeed.mach2tas(self.mach.value, temp = 'std', altitude =  self.press_alt_ft.value, temp_units='C', alt_units='ft', speed_units='kt')
        self.observers()
        
    def from_tas_kts(self, event = None):
        self.unobservers()
        self.cas_kts.value = aerocalc3.airspeed.tas2cas(self.tas_kts.value, self.press_alt_ft.value, temp='std', speed_units='kt', alt_units='ft', temp_units='C')
        self.mach.value = aerocalc3.airspeed.tas2mach(self.tas_kts.value, temp='std', altitude = self.press_alt_ft.value, temp_units='C', alt_units='ft', speed_units='kt')
        self.observers()
         
    def from_cas_kts(self, event=None):
        self.unobservers()
        self.tas_kts.value = aerocalc3.airspeed.cas2tas(self.cas_kts.value, self.press_alt_ft.value, temp='std', speed_units='kt', alt_units='ft', temp_units='C')
        self.mach.value = aerocalc3.airspeed.cas_alt2mach(self.cas_kts.value, self.press_alt_ft.value, speed_units='kt', alt_units='ft')
        self.observers()
    
            
spd = airspeed()

In [22]:
tab = Tab()
tab.set_title(0, "Pos")
tab.set_title(1, "Straight")
tab.set_title(2, "Turn")
tab.set_title(3, "ISA")
tab.set_title(4, "Speed")
tab.children = [VBox([geo_choice, geo_format, lat_input, lon_input, go_button]), \
                VBox([speed_kts, duration_sec, miniRange, maxiRange]), \
                VBox([gs_kts, bank_angle, radius_nm]), \
               isa.vbox(), \
               spd.vbox()]

m.add_control(WidgetControl(widget = tab, position="topright"))


In [23]:
# "Deg.dec","DegMin.dec","DegMinSec.dec" 
# "+/-DD.ddd" "N/SDDMM.ddd" "N/SDDMMSS.ddd"
# "+/-DDD.ddd" "W/EDDDMM.ddd" "W/EDDDMMSS.ddd"

r00 = IntSlider(value = 100, min = 10, max = 20000, step = 10, 
                 description = "Radius (m)", orientation='horizontal', readout=True, readout_format="d")

r00_circle = Circle(color = "blue", weight = 2, fill = False)

m.add_layer(r00_circle)

def r00_draw(event = None):
    if last_point_location:
        r00_circle.location = last_point_location
        r00_circle.radius = int(r00.value)

r00.observe(r00_draw, "value")  

r0 = Label(value="--- Square size (m) -------")
r0Lat = Text(placeholder="Dist (m)", description='Lat :', disabled=True)
r0Lon = Text(placeholder="Dist (m)", description='Lon :', disabled=True)

r1 = Label(value="--- Deg.dec -----------")
r1Lat = Text(placeholder="+/-DD.ddd", description='Lat :', disabled=True)
r1Lon = Text(placeholder="+/-DDD.ddd", description='Lon :', disabled=True)

r2 = Label(value="--- DegMin.dec --------")
r2Lat = Text(placeholder="N/SDDMM.ddd", description='Lat :', disabled=True)
r2Lon = Text(placeholder="W/EDDDMM.ddd", description='Lon :', disabled=True)

r3 = Label(value="--- DegMinSec.dec -----")
r3Lat = Text(placeholder="N/SDDMMSS.ddd", description='Lat :', disabled=True)
r3Lon = Text(placeholder="W/EDDDMMSS.ddd", description='Lon :', disabled=True)


result_control = WidgetControl(widget=VBox([r00, r0, r0Lat, r0Lon, r1, r1Lat, r1Lon, r2, r2Lat, r2Lon, r3, r3Lat, r3Lon]), position="topright")
m.add_control(result_control)

In [24]:
def format_lat_ddmm(value):
    if value < 0:
        h = "S"
    else:
        h = "N"
        
    value = abs(value)    
        
    d = int(abs(value))
    m = (value-d) * 60.0
    
    if m < 10:
        m_p = "0{}".format(m)  
    else:
        m_p = "{}".format(m)
    
    return "{}{:0>2d}{}".format(h, d, m_p)

def format_lon_ddmm(value):
    if value < 0:
        h = "W"
    else:
        h = "E"
    
    value = abs(value)   
    d = int(abs(value))
    m = (value-d) * 60.0
    
    if m < 10:
        m_p = "0{}".format(m)  
    else:
        m_p = "{}".format(m)
    
    return "{}{:0>3d}{}".format(h, d, m_p)

def format_lat_ddmmss(value):
    if value < 0:
        h = "S"
    else:
        h = "N"
    
    value = abs(value)   
    d = int(abs(value))
    m = (value-d) * 60.0
    mm = int(abs(m))
    s = (m-mm) * 60.0
    
    if s < 10:
        s_p = "0{}".format(s)  
    else:
        s_p = "{}".format(s)
    
    return "{}{:0>2d}{:0>2d}{}".format(h, d, mm, s_p)

def format_lon_ddmmss(value):
    if value < 0:
        h = "W"
    else:
        h = "E"
    
    value = abs(value)   
    d = int(abs(value))
    m = (value-d) * 60.0 
    mm = int(abs(m))
    s = (m-mm) * 60.0
    
    if s < 10:
        s_p = "0{}".format(s)  
    else:
        s_p = "{}".format(s)
    
    return "{}{:0>3d}{:0>2d}{}".format(h, d, mm, s_p)
    

In [25]:

def check_lat_dd(value):
    p = "^([+-]?)((\d{2}){1}(\.\d+)?)"
    # print(value)
    if re.fullmatch(p, value):
        tok = value.split('.')
        if abs(int(tok[0])) < 90:
            if len(tok) == 2:
                value_f = float(value)
                lower =  abs(float(tok[0])) + float( (int(tok[1])-1) ) / float( pow(10, len(str(tok[1]))))
                upper = abs(float(tok[0])) + float( (int(tok[1])+1) ) / float( pow(10, len(str(tok[1]))))
                if value_f < 0:
                    lower_f = -1.0 * upper
                    upper_f = -1.0 * lower
                else:
                    lower_f = lower
                    upper_f = upper
                return lower_f, value_f, upper_f
            elif len(tok) == 1:
                value_f = float(value)
                if value_f < 0:
                    lower_f = value_f-1
                    upper_f = value_f+1
                else:
                    lower_f = value_f-1
                    upper_f = value_f+1
                return lower_f, value_f, upper_f
            else:
                print("Number of tokens")
        else:
            print("Not in range")
    else:
        print("Format is not good")
    return None
            
def check_lon_dd(value):
    p = "^([+-]?)((\d{3}){1}(\.\d+)?)"
    # print(re.match(p, value))
    if re.fullmatch(p, value):
        tok = value.split('.')
        if abs(int(tok[0])) < 180:
            if len(tok) == 2:
                value_f = float(value)

                lower =  abs(float(tok[0])) + float( (int(tok[1])-1) ) / float( pow(10, len(str(tok[1]))))
                upper = abs(float(tok[0])) + float( (int(tok[1])+1) ) / float( pow(10, len(str(tok[1]))))

                if value_f < 0:

                    lower_f = -1.0 * upper
                    upper_f = -1.0 * lower
                else:
                    lower_f = lower
                    upper_f = upper

                return lower_f, value_f, upper_f
            elif len(tok) == 1:
                value_f = float(value)
                if value_f < 0:
                    lower_f = value_f-1
                    upper_f = value_f+1
                else:
                    lower_f = value_f-1
                    upper_f = value_f+1
                return lower_f, value_f, upper_f
    return None

def check_lat_ddmm(value):
    p = "^([NS]{1})((\d{2}){1}(\d{2}){1}(\.\d+)?)"
    # print(value)
    if re.fullmatch(p, value):
        tok = value.split('.')
        h = tok[0][0]
        if h == "N":
            h = 1.0
        elif h == "S":
            h = -1.0
            
        if (abs(int(tok[0][1:3])) < 90) and (abs(int(tok[0][3:5])) < 60):
            if len(tok) == 2:
                deg = tok[0][1:3]
                mn = tok[0][3:5] 
                dec = tok[1]
                
                mn_d = float(mn + '.' + dec) / 60.0
                value_f = h * (float(deg) + mn_d)
                
                mn_lower = float(mn) + float( (int(tok[1])-1) ) / float( pow(10, len(str(tok[1]))))
                mn_upper = float(mn) + float( (int(tok[1])+1) ) / float( pow(10, len(str(tok[1]))))
                
                lower_f = h * (float(deg) + mn_lower / 60.0)
                upper_f = h * (float(deg) + mn_upper / 60.0)
                
                return lower_f, value_f, upper_f
            
            elif len(tok) == 1:
                deg = tok[0][1:3]
                mn = tok[0][3:5] 
                
                mn_d = float(mn) / 60.0
                value_f = h * (float(deg) + mn_d)
                
                mn_lower = float(mn) - 1.0
                mn_upper = float(mn) + 1.0
                
                lower_f = h * (float(deg) + mn_lower / 60.0)
                upper_f = h * (float(deg) + mn_upper / 60.0)
                
                return lower_f, value_f, upper_f
            else:
                print("Number of tokens")
        else:
            print("Not in range")
    else:
        print("Format is not good")
    return None

def check_lon_ddmm(value):
    p = "^([WE]{1})((\d{3}){1}(\d{2}){1}(\.\d+)?)"
    # print(value)
    if re.fullmatch(p, value):
        tok = value.split('.')
        h = tok[0][0]
        if h == "E":
            h = 1.0
        elif h == "W":
            h = -1.0
            
        if (abs(int(tok[0][1:4])) < 180) and (abs(int(tok[0][4:6])) < 60) :
            if len(tok) == 2:
                deg = tok[0][1:4]
                mn = tok[0][4:6] 
                dec = tok[1]
                
                mn_d = float(mn + '.' + dec) / 60.0
                value_f = h * (float(deg) + mn_d)
                
                mn_lower = float(mn) + float( (int(tok[1])-1) ) / float( pow(10, len(str(tok[1]))))
                mn_upper = float(mn) + float( (int(tok[1])+1) ) / float( pow(10, len(str(tok[1]))))
                
                lower_f = h * (float(deg) + mn_lower / 60.0)
                upper_f = h * (float(deg) + mn_upper / 60.0)
                
                return lower_f, value_f, upper_f
            elif len(tok) == 1:
                deg = tok[0][1:4]
                mn = tok[0][4:6] 
                
                mn_d = float(mn) / 60.0
                value_f = h * (float(deg) + mn_d)
                
                mn_lower = float(mn) - 1.0
                mn_upper = float(mn) + 1.0
                
                lower_f = h * (float(deg) + mn_lower / 60.0)
                upper_f = h * (float(deg) + mn_upper / 60.0)
                return lower_f, value_f, upper_f
            else:
                print("Number of tokens")
        else:
            print("Not in range")
    else:
        print("Format is not good")
    return None

def check_lat_ddmmss(value):
    p = "^([NS]{1})((\d{2}){1}(\d{4}){1}(\.\d+)?)"
    # print(value)
    if re.fullmatch(p, value):
        tok = value.split('.')
        h = tok[0][0]
        if h == "N":
            h = 1.0
        elif h == "S":
            h = -1.0
            
        if (abs(int(tok[0][1:3])) < 90) and (abs(int(tok[0][3:5])) < 60) and (abs(int(tok[0][5:7])) < 60):
            if len(tok) == 2:
                deg = tok[0][1:3]
                mn = tok[0][3:5] 
                sec = tok[0][5:7]
                dec = tok[1]
                
                sec_d = float(sec + '.' + dec) / 60.0
                value_f = h * (float(deg) + (float(mn) + sec_d)/60.0)
                
                sec_lower = float(sec) + float( (int(tok[1])-1) ) / float( pow(10, len(str(tok[1]))))
                sec_upper = float(sec) + float( (int(tok[1])+1) ) / float( pow(10, len(str(tok[1]))))
                
                lower_f = h * (float(deg) + (float(mn) +(sec_lower / 60.0)) /60.0)
                upper_f = h * (float(deg) + (float(mn) +(sec_upper / 60.0)) /60.0)
                
                return lower_f, value_f, upper_f
            elif len(tok) == 1:
                deg = tok[0][1:3]
                mn = tok[0][3:5] 
                sec = tok[0][5:7]
                
                sec_d = float(sec) / 60.0
                value_f = h * (float(deg) + (float(mn) + sec_d)/60.0)
                
                sec_lower = float(sec) - 1.0
                sec_upper = float(sec) + 1.0
                
                lower_f = h * (float(deg) + (float(mn) +(sec_lower / 60.0)) /60.0)
                upper_f = h * (float(deg) + (float(mn) +(sec_upper / 60.0)) /60.0)
                return lower_f, value_f, upper_f
            else:
                print("Number of tokens")
        else:
            print("Not in range")
    else:
        print("Format is not good")
    return None

def check_lon_ddmmss(value):
    p = "^([WE]{1})((\d{3}){1}(\d{4}){1}(\.\d+)?)"
    # print(value)
    if re.fullmatch(p, value):
        tok = value.split('.')
        h = tok[0][0]
        if h == "E":
            h = 1.0
        elif h == "W":
            h = -1.0
            
        if (abs(int(tok[0][1:4])) < 180) and (abs(int(tok[0][4:6])) < 60) and (abs(int(tok[0][6:8])) < 60):
            if len(tok) == 2:
                deg = tok[0][1:4]
                mn = tok[0][4:6] 
                sec = tok[0][6:8]
                dec = tok[1]
                
                sec_d = float(sec + '.' + dec) / 60.0
                value_f = h * (float(deg) + (float(mn) + sec_d)/60.0)
                
                sec_lower = float(sec) + float( (int(tok[1])-1) ) / float( pow(10, len(str(tok[1]))))
                sec_upper = float(sec) + float( (int(tok[1])+1) ) / float( pow(10, len(str(tok[1]))))
                
                lower_f = h * (float(deg) + (float(mn) +(sec_lower / 60.0)) /60.0)
                upper_f = h * (float(deg) + (float(mn) +(sec_upper / 60.0)) /60.0)
                
                return lower_f, value_f, upper_f
            elif len(tok) == 1:
                deg = tok[0][1:4]
                mn = tok[0][4:6] 
                sec = tok[0][6:8]
                
                sec_d = float(sec) / 60.0
                value_f = h * (float(deg) + (float(mn) + sec_d)/60.0)
                
                sec_lower = float(sec) - 1.0
                sec_upper = float(sec) + 1.0
                
                lower_f = h * (float(deg) + (float(mn) +(sec_lower / 60.0)) /60.0)
                upper_f = h * (float(deg) + (float(mn) +(sec_upper / 60.0)) /60.0)
                return lower_f, value_f, upper_f
            else:
                print("Number of tokens")
        else:
            print("Not in range")
    else:
        print("Format is not good")
    return None

def check_lat_cursor(value):
    p = "^([+-]?)((\d{1,2}){1}(\.\d+)?)"
    # print(value)
    if re.fullmatch(p, value):
        tok = value.split('.')
        if abs(int(tok[0])) < 90:
            if len(tok) == 2:
                value_f = float(value)
                lower =  abs(float(tok[0])) + float( (int(tok[1])-1) ) / float( pow(10, len(str(tok[1]))))
                upper = abs(float(tok[0])) + float( (int(tok[1])+1) ) / float( pow(10, len(str(tok[1]))))
                if value_f < 0:
                    lower_f = -1.0 * upper
                    upper_f = -1.0 * lower
                else:
                    lower_f = lower
                    upper_f = upper
                return lower_f, value_f, upper_f
            elif len(tok) == 1:
                value_f = float(value)
                if value_f < 0:
                    lower_f = value_f-1
                    upper_f = value_f+1
                else:
                    lower_f = value_f-1
                    upper_f = value_f+1
                return lower_f, value_f, upper_f
            else:
                print("Number of tokens")
        else:
            print("Not in range")
    else:
        print("Format is not good")
    return None
            
def check_lon_cursor(value):
    p = "^([+-]?)((\d{1,3}){1}(\.\d+)?)"
    # print(re.match(p, value))
    if re.fullmatch(p, value):
        tok = value.split('.')
        if abs(int(tok[0])) < 180:
            if len(tok) == 2:
                value_f = float(value)

                lower =  abs(float(tok[0])) + float( (int(tok[1])-1) ) / float( pow(10, len(str(tok[1]))))
                upper = abs(float(tok[0])) + float( (int(tok[1])+1) ) / float( pow(10, len(str(tok[1]))))

                if value_f < 0:

                    lower_f = -1.0 * upper
                    upper_f = -1.0 * lower
                else:
                    lower_f = lower
                    upper_f = upper

                return lower_f, value_f, upper_f
            elif len(tok) == 1:
                value_f = float(value)
                if value_f < 0:
                    lower_f = value_f-1
                    upper_f = value_f+1
                else:
                    lower_f = value_f-1
                    upper_f = value_f+1
                return lower_f, value_f, upper_f
    return None

global cursor_marker
cursor_marker = None

global cursor_lat
cursor_lat = None
global cursor_lon
cursor_lon = None

def get_mouse_pos(**kwargs):
    global cursor_lat, cursor_lon
    global cursor_marker
    # print(kwargs)
    if geo_choice.value == "Cursor position" and kwargs["type"] == 'dblclick':
        # print(kwargs)
        if cursor_marker:
            m.remove_layer(cursor_marker)
            cursor_marker = None
        else:
            cursor_marker = Marker(location = kwargs["coordinates"], icon = AwesomeIcon(name='superpowers', marker_color='green', icon_color='darkgreen', spin=True))
            m.add_layer(cursor_marker)
            # m.add_layer(Marker(location = kwargs["coordinates"]))
            cursor_lat = kwargs["coordinates"][0]
            cursor_lon = kwargs["coordinates"][1]
            
    if geo_choice.value == "Cursor position":

        lat_input.value = str(kwargs["coordinates"][0])
        lon_input.value = str(kwargs["coordinates"][1])
        

            
m.on_interaction(get_mouse_pos)
    
    
def go_action(event):
    global last_point_location, cursor_lat, cursor_lon
    # print('--------------------------------------')
    # # "Deg.dec","DegMin.dec","DegMinSec.dec"
    # print(geo_choice.value)
    if geo_choice.value == "Cursor position":
        if cursor_marker:
            lat_input.value = str(cursor_marker.location[0])
            lon_input.value = str(cursor_marker.location[1])
        lat_l = check_lat_cursor(lat_input.value)
        lon_l = check_lon_cursor(lon_input.value)
    elif geo_choice.value == "Deg.dec" :
        lat_l = check_lat_dd(lat_input.value)
        lon_l = check_lon_dd(lon_input.value)
    elif geo_choice.value == "DegMin.dec" :
        lat_l = check_lat_ddmm(lat_input.value)
        lon_l = check_lon_ddmm(lon_input.value)
    elif geo_choice.value == "DegMinSec.dec" :
        lat_l = check_lat_ddmmss(lat_input.value)
        lon_l = check_lon_ddmmss(lon_input.value)
      
    # print(lat_l)
    # print(lon_l)
    if lat_l and lon_l:
        last_point_location = (lat_l[1], lon_l[1])
        _, _, dLat = geo.inv(lat_l[0], lon_l[0], lat_l[2], lon_l[0])
        _, _, dLon = geo.inv(lat_l[0], lon_l[0], lat_l[0], lon_l[2])
        r0Lat.value = str(dLat) 
        r0Lon.value = str(dLon) 
        r1Lat.value = str(lat_l[1]) 
        r1Lon.value = str(lon_l[1]) 
        r2Lat.value = format_lat_ddmm(lat_l[1])
        r2Lon.value = format_lon_ddmm(lon_l[1])
        r3Lat.value = format_lat_ddmmss(lat_l[1])
        r3Lon.value = format_lon_ddmmss(lon_l[1])
        
        marker = Marker(location=(lat_l[1], lon_l[1]), draggable=False)
        m.add_layer(marker)
        rectangle = Rectangle(bounds=((lat_l[0], lon_l[0]), (lat_l[2], lon_l[2])), weight = 1)
        m.add_layer(rectangle)
    else:
        last_point_location = None
        
    rangeProcessing()
    radius_computing()

go_button.on_click(go_action)

In [26]:
m


Map(center=[46.5, 6.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…